In [1]:
from filterpy.kalman import KalmanFilter
from sklearn.utils.linear_assignment_ import linear_assignment
import numpy as np

In [17]:
def convert_bbox_to_z(bbox):
  """
  Takes a bounding box in the form [x1,y1,x2,y2] and returns z in the form
    [x,y,s,r] where x,y is the centre of the box and s is the scale/area and r is
    the aspect ratio
  """
  w = bbox[2]-bbox[0]
  h = bbox[3]-bbox[1]
  x = bbox[0]+w/2.
  y = bbox[1]+h/2.
  s = w*h    #scale is just area
  r = w/float(h)
  return np.array([x,y,s,r]).reshape((4,1))

def convert_x_to_bbox(x,score=None):
  """
  Takes a bounding box in the centre form [x,y,s,r] and returns it in the form
    [x1,y1,x2,y2] where x1,y1 is the top left and x2,y2 is the bottom right
  """
  w = np.sqrt(x[2]*x[3])
  h = x[2]/w
  if(score==None):
    return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.]).reshape((1,4))
  else:
    return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.,score]).reshape((1,5))

def init_kalman(bbox):
    kf = KalmanFilter(dim_x=7, dim_z=4)
    kf.F = np.array([[1,0,0,0,1,0,0],[0,1,0,0,0,1,0],[0,0,1,0,0,0,1],[0,0,0,1,0,0,0],  [0,0,0,0,1,0,0],[0,0,0,0,0,1,0],[0,0,0,0,0,0,1]])
    kf.H = np.array([[1,0,0,0,0,0,0],[0,1,0,0,0,0,0],[0,0,1,0,0,0,0],[0,0,0,1,0,0,0]])

    kf.R[2:,2:] *= 10.
    kf.P[4:,4:] *= 1000. #give high uncertainty to the unobservable initial velocities
    kf.P *= 10.
    kf.Q[-1,-1] *= 0.01
    kf.Q[4:,4:] *= 0.01
    kf.x[:4] = convert_bbox_to_z(bbox)
    return kf

def update(k, bbox):
    k.update(convert_bbox_to_z(bbox))
    if((k.x[6]+k.x[2])<=0):
        k.x[6] *= 0.0
    k.predict()
    print(convert_x_to_bbox(k.x))
    return k

In [32]:
eg_bbox_0 = np.array([1,2,3,4])
eg_bbox_1 = np.array([2,3,4,5])
eg_bbox_2 = np.array([3,4,5,6])
eg_bbox_3 = np.array([4,5,6,7])
eg_bbox_4 = np.array([5,6,7,8])
k = init_kalman([1,2,3,4])
print(convert_x_to_bbox(k.x))

[[1. 2. 3. 4.]]


In [33]:
k1 = update(k,np.array([2,3,4,5]))

[[1.90909091 2.90909091 3.90909091 4.90909091]]


In [41]:
print(k)

KalmanFilter object
dim_x = 7
dim_z = 4
dim_u = 0
x = [[11.01525988 12.01525988  4.          1.          1.00947188  1.00947188.T
P = [[2.11974368 0.         0.         0.         0.31665898 0.
      0.        ]
     [0.         2.11974368 0.         0.         0.         0.31665898
      0.        ]
     [0.         0.         7.46174712 0.         0.         0.
      1.12828865]
     [0.         0.         0.         3.72220852 0.         0.
      0.        ]
     [0.31665898 0.         0.         0.         0.21148179 0.
      0.        ]
     [0.         0.31665898 0.         0.         0.         0.21148179
      0.        ]
     [0.         0.         1.12828865 0.         0.         0.
      0.34061377]]
F = [[1 0 0 0 1 0 0]
     [0 1 0 0 0 1 0]
     [0 0 1 0 0 0 1]
     [0 0 0 1 0 0 0]
     [0 0 0 0 1 0 0]
     [0 0 0 0 0 1 0]
     [0 0 0 0 0 0 1]]
Q = [[1.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
     [0.e+00 1.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
     [0.e+00 0.e+00

In [34]:
k = update(k,np.array([3,4,5,6]))

[[4.09048277 5.09048277 6.09048277 7.09048277]]


In [35]:
k = update(k,np.array([4,5,6,7]))

[[5.05724513 6.05724513 7.05724513 8.05724513]]


In [36]:
k = update(k,np.array([5,6,7,8]))

[[6.04010397 7.04010397 8.04010397 9.04010397]]


In [37]:
k = update(k,np.array([6,7,8,9]))

[[ 7.02981672  8.02981672  9.02981672 10.02981672]]


In [38]:
k = update(k,np.array([7,8,9,10]))

[[ 8.02313144  9.02313144 10.02313144 11.02313144]]


In [39]:
k = update(k,np.array([8,9,10,11]))

[[ 9.01854576 10.01854576 11.01854576 12.01854576]]


In [40]:
k = update(k,np.array([9,10,11,12]))

[[10.01525988 11.01525988 12.01525988 13.01525988]]
